# 작업형1

## 문제1
각 구급 보고서별 출동시간과 신고시각의 차이를 '소요시간' 컬럼으로 만들어 초(sec) 단위로 구하고

소방서명별 소요시간의 평균을 오름차순으로 정렬했을 때 3번째로 작은 소요시간의 값과 소방서명을 출력

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/bigdata-csvfile/report_data.csv')

df['신고일시'] = pd.to_datetime(
   df['신고일자'].astype(str) + df['신고시각'].astype(str).str.zfill(6),
    format = '%Y%m%d%H%M%S')

df['출동일시'] = pd.to_datetime(
   df['출동일자'].astype(str) + df['출동시각'].astype(str).str.zfill(6),
    format = '%Y%m%d%H%M%S')
    
df['소요시간'] = (df['출동일시'] - df['신고일시']).dt.total_seconds()

df2 = df.groupby('소방서명')['소요시간'].mean().reset_index().sort_values(by='소요시간')
print(df2.iloc[2])

소방서명    종로소방서
소요시간    175.5
Name: 21, dtype: object


## 문제2

학교 세부 유형이 일반중학교인 학교들 중 일반중학교 숫자가 2번째로 많은 시도의 일반중학교 데이터만 필터하여 해당 시도의 교원 한 명당 맡은 학생 수가 가장 많은 학교를 찾아서 해당 학교의 교원 수를 출력하시오.

(단, 값이 0 또는 NaN의 경우 값이 없는 것으로 판단하여 연산에서 제외한다.)

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/bigdata-csvfile/class_data.csv')

# 일반중학교가 2번째로 많은 시도
df = df[df['학교세부유형'] == '일반중학교']
# print(df['시도'].value_counts()) #서울    387

# 맡은 학생 수가 가장 많은 학교의 교원 수
df = df[df['시도'] == '서울']
new_data = df.groupby(['시도', '학교명', '학교세부유형']).sum()
new_data['cond'] = new_data['일반학급_학생수_계'] / new_data['교원수_총계_계']
new_data = new_data.sort_values(by='cond', ascending=False)
result = new_data.iloc[0]
print(result['교원수_총계_계']) 

33.0


## 문제3

5대 범죄(절도, 사기, 배임, 방화, 폭행)의 월별 총발생건수를 총범죄수라고 표현 하자. 

2018, 2019년의 각각 분기별 총범죄수의 월평균 값을 구했을 때 최댓값을 가지는 연도와 분기를 구하고 해당 분기의 최댓값의 사기 범죄가 발생한 월의 사 기 범죄 발생건수를 출력하시오. 

(단, 분기 정보는 다음과 같이 한다.
1분기 :1, 2, 3월 / 2분기 : 4, 5, 6월 / 3분기 : 7, 8, 9월 / 4분기 : 10, 11, 12월, 

- 1분기 월평균 : 1, 2, 3월의 총범죄수 평균, 2분기 월평균 : 4, 5, 6월의 총범죄수 평균,
- 3분기 월평균 : 7, 8, 9월의 총범죄수 평균, 4분기 월평균 : 10, 11, 12월의 총범 죄수 평균)

In [3]:
# 방법1
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/bigdata-csvfile/crime_data.csv')
df['총발생건수'] = df['절도'] + df['사기'] + df['배임'] + df['방화'] + df['폭행']

brach_2018_1 = df[0:3]['총발생건수'].sum()
brach_2018_2 = df[3:6]['총발생건수'].sum()
brach_2018_3 = df[6:9]['총발생건수'].sum()
brach_2018_4 = df[9:12]['총발생건수'].sum()

brach_2019_1 = df[12:15]['총발생건수'].sum()
brach_2019_2 = df[15:18]['총발생건수'].sum()
brach_2019_3 = df[18:21]['총발생건수'].sum()
brach_2019_4 = df[21:24]['총발생건수'].sum()

list = [brach_2018_1, brach_2018_2, brach_2018_3, brach_2018_4, brach_2019_1, brach_2019_2, brach_2019_3, brach_2019_4]

print(list.index(max(list))) #brach_2019_2

print(df[15:18]['사기'].max()) #27766

5
27766


In [4]:
# 방법2
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/bigdata-csvfile/crime_data.csv')

#2018년 2019년 각각 분기별 총범죄자수 월평균 값

df['범죄분류'] = pd.to_datetime(df['범죄분류'], format='%Y년_%m월')
df['범죄분류'] = df['범죄분류'].dt.to_period('M')
df['연도'] = df['범죄분류'].dt.year
df['월'] = df['범죄분류'].dt.month

def branch(number):
    if number in [1, 2, 3]:
        return '1분기'
    elif number in [4, 5, 6]:
        return '2분기'
    elif number in [7, 8, 9]:
        return '3분기'
    else:
        return '4분기'

df['분기'] = df['월'].apply(branch)
df['총발생건수'] = df['절도'] + df['사기'] + df['배임'] + df['방화'] + df['폭행']

# 최댓값을 가지는 연도와 분기를 구하고 해당 분기의 최댓값의 사기 범죄가 발생한 월의 사기 범죄 발생건수를 출력하시오.
df2 = df.groupby(['분기', '연도'])['총발생건수'].sum().reset_index().sort_values(by='총발생건수', ascending=False)
print(df2.iloc[0]) # 2019년도 2분기 총발생건수 175315

# 2019-05 2분기의 사기범죄 발생건수
cond = df.loc[(df['연도']==2019) & (df['월']==5) & (df['분기'] == '2분기'),'사기'].max()
print(cond) #27766



분기          2분기
연도         2019
총발생건수    175315
Name: 3, dtype: object
27766


## 작업형2


주어지는 데이터를 활용하여 예측 변수 General_Health에 대해 ID별로 GeneralLHealth 값을 예측하여 제출한다. 

(단, 제출 데이터 칼럼은 ID와 General_Health 두 개만 존재해야 하고, 평가지표는 macro f1 score로 한다. 제 출 파일 이름은 6th_test_type2.csv로 한다.)

In [5]:
import pandas as pd
import numpy as np

train = pd.read_csv('/kaggle/input/bigdata-csvfile/test6_type2_train.csv')
test = pd.read_csv('/kaggle/input/bigdata-csvfile/test6_type2_test.csv')
y = train['General_Health']

train = train.drop(columns = 'General_Health')
# 필요없는 컬럼 제거
train_drop = train.drop(columns = 'ID')
test_drop = test.drop(columns = 'ID')

# 컬럼개수 확인
print(train.shape, test.shape)

# 인코딩
train_dummies = pd.get_dummies(train_drop)
test_dummies = pd.get_dummies(test_drop)

# 컬럼개수 맞추기
for col in train_dummies.columns:
    if col not in test_dummies.columns:
        train_dummies.drop(columns=col, axis=1, inplace=True)

for col in test_dummies.columns:
    if col not in train_dummies.columns:
        test_dummies.drop(columns=col, axis=1, inplace=True)


# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(train_dummies, y)
pred = model.predict(test_dummies)

# 데이터프레임 생성
result = pd.DataFrame({'pred' : pred})
# csv
result.to_csv('result.csv', index=False)
csv_read = pd.read_csv('result.csv')
print(csv_read)

(11230, 19) (4813, 19)
           pred
0     Excellent
1     Excellent
2          Good
3     Excellent
4     Excellent
...         ...
4808  Excellent
4809       Good
4810       Fair
4811       Fair
4812       Good

[4813 rows x 1 columns]


# 작업형3

## 문제1

S 도시에는 남성 700명과 여성 500명이 있다. S 도시의 남성 중 기혼자의 비율은 30%이고, 여성 중 기혼자의 비율은 20%이다. 남성과 여성의 혼인 여부에 따른 인 구 비율이 동일한지 확인해본다. 각 문항의 답을 제출 형식에 맞춰 제출하시오.

1. 남, 여 기혼자와 미혼자에 대한 카이제곱 검정을 진행하고, 카이제곱 통계량을 출력 한다.
2. 유의수준 0.05를 기준으로 할 때, 카이제 검정 결과의 p_value를 출력한다.
3. 검증된 p_ value 결과를 바탕으로 '귀무가 기각' 혹은 귀무가 채택'을 출력한다.

In [6]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.DataFrame({'남자': [700 * 0.3, 700 * 0.7], '여자':[500 * 0.2, 500*0.8]},index = ['기혼자', '비혼자'])

# 남녀 기혼자 카이제곱 검정
st, pv, dof, exp_f = stats.chi2_contingency(df)
print(st, pv)

if pv < 0.05 :
    print('귀무가설 기각 남성과 여성의 혼인 여부에 따른 인구 비율이 동일하지 않다.')
else:
    print('귀무가설 채택 남성과 여성의 혼인 여부에 따른 인구 비율이 동일하다.')

14.705452286024956 0.00012568242320118184
귀무가설 기각 남성과 여성의 혼인 여부에 따른 인구 비율이 동일하지 않다.


## 문제2

주어진 건강정보 데이터를 활용하여 다중 선형 회귀 분석을 하고자 한다. 각 문항의 답을 제출 형식에 맞춰 제출하시오.

1. age Cholesterol 칼럼으로 weight를 예측하는 선형 회귀 모델을 만들려고 한다.
age의 회귀계수를 구하시오. (단, 반올림하여 소수점 3자리까지 입력한다.)
2. age가 45이고, Cholesterol이 67.8일 때 문제 1의 모델을 기반으로 Weight 값을 예측하시오.
3. 문제 1의 선형 회귀 모델에서 유의수준 0.05일 때 Cholesterol 회귀계수의 신뢰구 간을 구하시오.

In [7]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.formula.api import ols

df = pd.read_csv('/kaggle/input/bigdata-csvfile/cholesterol_data.csv')
formula = ('weight ~ age + Cholesterol')
model = ols(formula, data=df).fit()
print(round(model.params['age'],3))

new_data = pd.DataFrame({'const':[1], 'age':[45], 'Cholesterol':[67.9]})
result = model.predict(new_data)
print(round(result[0], 3))

print(model.summary())
print('0.039 ~ 0.125')

-0.036
78.834
                            OLS Regression Results                            
Dep. Variable:                 weight   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     7.574
Date:                Tue, 10 Jun 2025   Prob (F-statistic):           0.000608
Time:                        20:55:43   Log-Likelihood:                -1374.9
No. Observations:                 333   AIC:                             2756.
Df Residuals:                     330   BIC:                             2767.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      74.8953      4.455   